In [1]:
import importlib
import def_save

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
importlib.reload(def_save)
from def_save import *

In [3]:
df1 = pd.read_csv('../../Downloads/clustering_2.csv')

In [4]:
# long beach location
df2 = port_5km(df1, 33.754929, -118.214344)
df3 = representative_value(df2)

### clutering

In [5]:
df4 = auto_clustering(df3, 8)
df5 = re_cluster(df4, 42)

In [6]:
# u = pd.merge(df1, df5[['MMSI', 'cluster_prediction']], how='left', on='MMSI').dropna().reset_index(drop=True)
# folium_save(u)

### train, test data split(Split by : 2018 year)

In [7]:
train_data, test_data = train_test_split(df2)

In [8]:
train_data_rep = representative_value(train_data.iloc[:, :5])

In [9]:
train_data_cluster = auto_clustering(train_data_rep, 8)
train_data_cluster2 = re_cluster(train_data_cluster, 42)

In [10]:
# u = pd.merge(df1, train_data_cluster2[['MMSI', 'cluster_prediction']], how='left', on='MMSI').dropna().reset_index(drop=True)
# folium_save(u)

##### good clustering -> baseline selection of train ship -> test ship search route for baseline -> 
##### Learning with ships having similar route
#####  

In [11]:
train_data_rep = representative_value(train_data)

In [12]:
idx = list(set(list(range(0,train_data_rep.shape[1])))-set(list(range(0,train_data_rep.shape[1],3))))
idx.extend([train_data_rep.columns.tolist().index('MMSI'), list(range(0,train_data_rep.shape[1],3))[-3]])

In [13]:
train_data_rep = train_data_rep.iloc[:, idx].drop(columns=['rpst_value_LAT8', 'rpst_value_LON8', 'rpst_value_LAT9', 'rpst_value_LON9'])

In [14]:
train_data_rep = pd.merge(train_data_rep, train_data_cluster2[['MMSI', 'cluster_prediction']], how='left', on='MMSI')

In [15]:
test_data_rep = representative_value(test_data)

In [16]:
idx = list(set(list(range(0,test_data_rep.shape[1])))-set(list(range(0,test_data_rep.shape[1],3))))
idx.extend([test_data_rep.columns.tolist().index('MMSI'), list(range(0,test_data_rep.shape[1],3))[-3]])

In [17]:
test_data_rep = test_data_rep.iloc[:, idx].drop(columns=['rpst_value_LAT8', 'rpst_value_LON8', 'rpst_value_LAT9', 'rpst_value_LON9'])

In [18]:
test_data_rep = route_similar(train_data_rep, test_data_rep)

##### test data 유사 군집 종류와 개수 확인

In [19]:
pd.value_counts(test_data_rep['cluster_prediction'])

3001    4
100     2
72      1
Name: cluster_prediction, dtype: int64

#### 3001 cluster에 속한 predict ship이 많으므로 3001만 먼저 해본다.

In [20]:
cluster_name = '3001'

In [21]:
train = train_data_rep[train_data_rep['cluster_prediction']==cluster_name][['MMSI', 'y_value7']]

In [22]:
test = test_data_rep[test_data_rep['cluster_prediction']==cluster_name][['MMSI', 'y_value7']]

### latent variable - Data_metrization
###### effect dimensional reduction + baseline's scheduled time for last point 
######  

In [23]:
train_final = Dmetrization(pd.read_csv('../../Downloads/clustering_2.csv'), train) # raw data path

In [24]:
test_final = Dmetrization(pd.read_csv('../../Downloads/clustering_2.csv'), test) # raw data path

In [25]:
test_final = test_receive_t1t7(train_final, test_final)

In [26]:
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [33]:
y_pred = xgb.XGBRegressor(booster='gbtree', 
                          eta=0.01, 
                          scale_pos_weight=0.1,
                          min_child_weight=13,
                          colsample_bylevel=0.3,
                          colsample_bytree=0.8,
                          subsample=1,
                          max_depth=8, 
                          gamma=0.1, 
                          alpha=0.1,
                          seed=1028,
                          n_estimators=1000).fit(
    scale(train_final.drop(columns=['y_value'])), train_final['y_value']).predict(scale(test_final.drop(columns=['y_value'])))
y_pred = np.round(y_pred, 1).astype(str)

In [34]:
np.round(pd.DataFrame({'prediction': y_pred, 'true': test_final['y_value']}), 1).astype(float)
# np.mean((np.round(pd.DataFrame({'prediction': y_pred, 'true': test_final['y_value']}), 1).astype(float).iloc[:,1]-np.round(pd.DataFrame({'prediction': y_pred, 'true': test_final['y_value']}), 1).astype(float).iloc[:,0])**2)

,prediction,true
0,84.3,76.8
1,83.9,84.9
2,88.4,93.8
3,84.2,90.0


### grid search

In [29]:
from tqdm import tqdm

In [40]:
mse = 100
new_mse = 0
L = []
for reg_lambda in tqdm([1/20 for i in list(range(1, 11))]):
    for colsample_bylevel in [i/20 for i in list(range(1, 11))]:
        for colsample_bytree in [i/20 for i in list(range(1, 11))]:
            for subsample in [i/20 for i in list(range(1, 11))]:
                for max_delta_step in [i/20 for i in list(range(1, 11))]:
                    for min_child_weight in list(range(1, 16)):
                        for max_depth in list(range(1, 8)):
                            y_pred = xgb.XGBRegressor(booster='gbtree', 
                                      max_delta_step=max_delta_step,
                                      scale_pos_weight = 1,
                                      eta=0.01, 
                                      min_child_weight=min_child_weight,
                                      max_depth=max_depth, 
                                      reg_lambda=reg_lambda,
                                      gamma=0.1, 
                                      alpha=0.1, 
                                      subsample=subsample,
                                      colsample_bylevel = colsample_bylevel,
                                      colsample_bytree = colsample_bytree,seed = 1028,
                                      n_estimators=1000).fit(
                (train_final.drop(columns=['y_value'])), train_final['y_value']).predict((test_final.drop(columns=['y_value'])))
                            y_pred = np.round(y_pred, 1).astype(str)
                            new_mse = np.mean((np.round(pd.DataFrame({'prediction': y_pred, 'true': test_final['y_value']}), 1).astype(float).iloc[:,1]-np.round(pd.DataFrame({'prediction': y_pred, 'true': test_final['y_value']}), 1).astype(float).iloc[:,0])**2)
                    
                            if new_mse < mse:
                                mse = new_mse
                                L = [reg_lambda, colsample_bylevel, colsample_bytree, subsample, max_delta_step, min_child_weight, max_depth]

100%|███████████████████████████████████████| 10/10 [1:06:00<00:00, 391.12s/it]


In [41]:
mse

44.47500000000002

In [42]:
L

[0.1, 0.4, 0.8, 1.0, 1.0]

In [49]:
importlib.reload(def_save)
from def_save import *